In [1]:
from happytransformer import HappyTextToText

C:\Myfiles\Apps\envs\asl\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
happy_tt = HappyTextToText("T5", "t5-base")

C:\Myfiles\Apps\envs\asl\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
04/11/2023 01:00:01 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [3]:
import torch

In [4]:
import os
dir = os.getcwd()
filename = "model.pth"
filepath = os.path.join(os.getcwd(), filename)
torch.save(happy_tt, filepath)

In [5]:
from datasets import load_dataset


In [6]:
train_dataset = load_dataset("jfleg", split='validation[:]')


In [7]:
eval_dataset = load_dataset("jfleg", split='test[:]')


In [8]:
eval_dataset

Dataset({
    features: ['sentence', 'corrections'],
    num_rows: 748
})

In [9]:
model = torch.load("new_model.pth")

In [10]:
model

In [11]:
for case in train_dataset["corrections"][:2]:
  print(case)
  print(case[0])
  print("--------------------------------------------------------")

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
--------------------------------------------------------
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 
--------------------------------------------------------


In [12]:
import csv

def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the task's prefix to input 
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases contain blank strings. 
                if input_text and correction:
                    writter.writerow([input_text, correction])
                    


generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)
              

In [ ]:
 before_result = happy_tt.eval("eval.csv")

In [ ]:
print("Before loss:", before_result.loss)


In [ ]:
from happytransformer import TTTrainArgs

In [9]:
args = TTTrainArgs(batch_size=4)
model.train("train.csv", args=args)

Found cached dataset csv (C:/Users/Dhiyanesh/.cache/huggingface/datasets/csv/default-cbfcfeb82bed5e24/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Map:   0%|                                                                             | 0/3016 [00:00<?, ? examples/s]C:\Myfiles\Apps\envs\asl\lib\site-packages\transformers\tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
C:\Myfiles\Apps\envs\asl\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warning

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
import os
dir = os.getcwd()
filename = "new_model.pth"
filepath = os.path.join(os.getcwd(), filename)
torch.save(model, filepath)

In [41]:
before_loss = model.eval("eval.csv")

04/11/2023 01:58:52 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 74.71it/s]
PyTorch: setting up devices                                                                                            
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


In [42]:
print("After loss: ", before_loss.loss)

After loss:  0.450774610042572


In [43]:
from happytransformer import TTSettings
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)

In [39]:
example_1 = "grammar:how you doing"
result_1 = model.generate_text(example_1, args=beam_settings)
print(result_1.text)

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



How you are doing?
